
---

# 🔐 **Dependency Injection in FastAPI**

> **Goal:** Reuse logic like **DB connections**, **auth checks**, or **configs** across routes — using FastAPI’s built-in `Depends()` function.

---

## 💡 What is `Depends()`?

Use `Depends()` to **inject the return value of a function** into your API route — no need to call it manually.

```python
from fastapi import FastAPI, Depends

app = FastAPI()

# ✅ This function can be reused across routes
def say_hello():
    return "Hello from dependency!"

# Inject the result into this route
@app.get("/greet")
def greet(message: str = Depends(say_hello)):
    return {"msg": message}
```

🔸 **Benefits:**

* Avoid repeated code
* Easy to swap logic during tests
* Included in API docs automatically

---

## 🔁 Shared Dependencies (Apply to Multiple Routes)

Use `dependencies=[Depends(...)]` to **enforce logic without using its return value**.

```python
from fastapi import Header, HTTPException, Depends

# ✅ Token validator logic
def verify_token(x_token: str = Header(...)):
    if x_token != "valid-token":
        raise HTTPException(status_code=403, detail="Forbidden")

# ✅ Inject into multiple routes
@app.get("/profile", dependencies=[Depends(verify_token)])
def profile_data():
    return {"user": "authorized"}
```

📌 Good for:

* Auth checks
* Logging
* Rate limiting

---

## 🪜 Sub-Dependencies (Depends Inside Depends)

Dependencies can be **layered** — one depends on another.

```python
# Step 1: DB connection logic
def get_db():
    return "DB connection"

# Step 2: Get user using DB
def get_user(db=Depends(get_db)):
    return f"User from → {db}"

# Step 3: Inject user into route
@app.get("/me")
def read_user(user=Depends(get_user)):
    return {"user": user}
```

🔄 FastAPI resolves them **in order** — so `/me` calls:

1. `get_db()`
2. then `get_user()`
3. then returns the user

---

## 🧱 Class-Based Dependencies

For **complex logic**, you can create a class with `__call__()` and use it like a function.

```python
from fastapi import Header, HTTPException

# ✅ AuthService as a class
class AuthService:
    def __init__(self, token: str = Header(...)):
        self.token = token

    def __call__(self):
        if self.token != "admin-token":
            raise HTTPException(status_code=401, detail="Unauthorized")
        return self.token

# Inject the class instance
@app.get("/admin")
def get_admin_data(token: str = Depends(AuthService())):
    return {"access": "Admin granted"}
```

📌 Good for:

* Business logic layers
* Services like DBClient, AuthService, etc.
* When you need internal state or configuration

---

## ✅ Summary Table

| 🔹 Feature        | 🔍 What It Does                  | 🧩 Real Use Case                |
| ----------------- | -------------------------------- | ------------------------------- |
| `Depends()`       | Inject a function’s return value | Reuse any logic (e.g. greeting) |
| Shared Dependency | Run logic across many routes     | Auth, logging, validation       |
| Sub-Dependencies  | Nest one dependency into another | Get user from DB, token auth    |
| Class-Based       | Use class as dependency          | AuthService, Config, Payments   |

---
